IMPORTING LIBRARIES

In [1]:
from langchain_groq import ChatGroq

IMPORTING FILE

In [2]:
with open("key.txt", "r") as f:
    api_key=f.readline().strip()

INITIALIZE LLM MODEL

In [3]:
llm = ChatGroq(temperature=0, groq_api_key=api_key, model_name="llama-3.1-70b-versatile")

WEB SCRAPING

In [4]:
from langchain_community.document_loaders import WebBaseLoader


def loading_web(url):
    loader = WebBaseLoader(url)
    page_data=loader.load()
    print(page_data)
    return page_data

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
data=loading_web("https://walmart.wd5.myworkdayjobs.com/en-US/WalmartExternal/job/IN-KA-BANGALORE-Home-Office-Building-11/SOFTWARE-ENGINEER-III_R-2049883-1")

[Document(metadata={'source': 'https://walmart.wd5.myworkdayjobs.com/en-US/WalmartExternal/job/IN-KA-BANGALORE-Home-Office-Building-11/SOFTWARE-ENGINEER-III_R-2049883-1', 'title': '', 'description': "Position Summary... Demonstrates up-to-date expertise and applies this to the development, execution, and improvement of action plans by providing expert advice and guidance to others in the application of information and best practices; supporting and aligning efforts to meet customer and business needs; and building commitment for perspectives and rationales. Provides and supports the implementation of business solutions by building relationships and partnerships with key stakeholders; identifying business needs; determining and carrying out necessary processes and practices; monitoring progress and results; recognizing and capitalizing on improvement opportunities; and adapting to competing demands, organizational changes, and new responsibilities. Models compliance with company policie

In [13]:
job_details=data[0].metadata["description"]

In [14]:
from langchain_core.prompts import PromptTemplate

prompts_extract =PromptTemplate(
""" 
### SCRAPED TEXT FROM WEBSITE:
{page_data}
### INSTRUCTION:
The scraped text is from the career's page of a wesite.
Your job is to extract the job postings and return them in JSON format containing 
following keys : `role`, `experience`, `skills` and `description`.
Only return the valid json
### VALID JSON (NO PREAMBLE):
"""
)

chain_extract = prompt_extract | llm 

TypeError: BaseModel.__init__() takes 1 positional argument but 2 were given